In [48]:
import requests
from tqdm import tqdm
import numpy as np
from astropy.io import ascii

In [79]:
field_apass_path = '../Part II - nova calibration/Data/A4.txt'

field_apass_spec = np.dtype({'names': [
    'Name', 'RA', 'Dec', 'B mag', 'B mag err',
    'V mag', 'V mag err',
    'g mag', 'g mag err',
    'r mag', 'r mag err',
    'i mag', 'i mag err'
    ], 'formats': [
    'U16', float, float, float, float,
    float, float, float, float, float, float, float, float
    ]})
field_apass = np.loadtxt(field_apass_path, dtype=field_apass_spec)

In [96]:
def get_panstarrs_matches_unc(field_stars, radius):
    '''
    WIP

    radius in arcseconds
    '''
    radius = radius/3600.

    columns = """objID,raMean,decMean,nDetections,ng,nr,ni,nz,ny,
        gMeanPSFMag,rMeanPSFMag,iMeanPSFMag,zMeanPSFMag,yMeanPSFMag,
        gMeanPSFMagErr,rMeanPSFMagErr,iMeanPSFMagErr,zMeanPSFMagErr,yMeanPSFMagErr""".split(',')
    columns = [x.strip() for x in columns]
    columns = [x for x in columns if x and not x.startswith('#')]
    results = []
    constraints = {'nDetections.gt':1}
    for i in tqdm(range(len(field_stars))):
        data = constraints.copy()
        data['ra'] = field_stars[i]['RA']
        data['dec'] = field_stars[i]['Dec']
        data['radius'] = radius
        results += [ps1search(table='mean',release='dr2',format='csv',columns=columns, **data)]
    out = clean_results(results)
    return out
    


def ps1search(table="mean",release="dr2",format="csv",columns=None,
           baseurl="https://catalogs.mast.stsci.edu/api/v0.1/panstarrs", verbose=False,
           **kw):
    """Do a general search of the PS1 catalog (possibly without ra/dec/radius)
    
    Parameters
    ----------
    table (string): mean, stack, or detection
    release (string): dr1 or dr2
    format: csv, votable, json
    columns: list of column names to include (None means use defaults)
    baseurl: base URL for the request
    verbose: print info about request
    **kw: other parameters (e.g., 'nDetections.min':2).  Note this is required!
    """
    
    data = kw.copy()
    url = f"{baseurl}/{release}/{table}.{format}"
    # two different ways to specify a list of column values in the API
    # data['columns'] = columns
    data['columns'] = '[{}]'.format(','.join(columns))

    # either get or post works
    # r = requests.post(url, data=data)
    print(url)
    print(data)
    r = requests.get(url, params=data)

    r.raise_for_status()
    if format == "json":
        return r.json()
    else:
        return r.text
    
def clean_results(results):
    out = []
    for i in range(len(results)):
        if len(results[i]) > 0:
            tab = ascii.read(results[i])
            # improve the format
            for filter in 'grizy':
                col = filter+'MeanPSFMag'
                try:
                    tab[col].format = ".4f"
                    tab[col][tab[col] == -999.0] = np.nan
                except KeyError:
                    print("{} not found".format(col))
        else:
            tab = []
        out += [tab]
    out_filtered = out
    for i in range(len(out)):
        inds = []
        if out[i] != []:
            outarr = out[i].to_pandas().to_numpy()
            for j in range(len(outarr)):
                if np.any(np.isnan(outarr[j])) == False:
                    inds += [j]
            out_filtered[i] = outarr[inds]
    return out_filtered

In [151]:
field_stars = field_apass[0:4]

In [199]:
columns = """objID,raMean,decMean,nDetections,ng,nr,ni,nz,ny,
    gMeanPSFMag,rMeanPSFMag,iMeanPSFMag,zMeanPSFMag,yMeanPSFMag,
    gMeanPSFMagErr,rMeanPSFMagErr,iMeanPSFMagErr,zMeanPSFMagErr,yMeanPSFMagErr""".split(',')
columns = [x.strip() for x in columns]
columns = [x for x in columns if x and not x.startswith('#')]
data = {}

data['radius'] = 2/3600
data['columns'] = '[{}]'.format(','.join(columns))
data['nDetections.gt'] = 1
url = "https://catalogs.mast.stsci.edu/api/v0.1/panstarrs/dr2/mean.csv"
results = []
for f in field_stars:
    data['ra'] = f['RA']
    data['dec'] = f['Dec']
    r = requests.get(url, params=data)
    r.raise_for_status()
    if r.text == '':
        tab = []
    else:
        tab = ascii.read(r.text)
        for f in 'grizy':
            col = f + 'MeanPSFMag'
            tab[col][tab[col] == -999.0] = np.nan
            col = f + 'MeanPSFMagErr'
            tab[col][tab[col] == -999.0] = np.nan
    results += [tab]